# Volt Analytics

## Import Dependencies

In [1]:
import os
import datetime as dt
import numpy as np
import pandas as pd
import boto3

boto3.setup_default_session(region_name='us-east-1')
s3 = boto3.resource('s3')

pd.set_option('display.max_rows', 500)

In [2]:
currentDate =  dt.datetime.today().date()
currentDate

datetime.date(2018, 2, 22)

## Read in CSV files

In [3]:
#Volt Daily Data
voltDailyData = pd.read_csv('data/voltdailydata_' + currentDate.strftime('%Y%m%d') + '.csv')
voltDailyData['Date'] = pd.to_datetime(voltDailyData['Date'])
voltDailyData['Date'] = voltDailyData['Date'].dt.date

In [4]:
voltDailyData.head()

,Date,MilesDriven,EvMilesDriven,GallonsBurned
0,2017-06-25,163.7,52.6,3.03005
1,2017-06-26,202.4,42.5,4.15543
2,2017-06-27,202.3,53.8,3.97314
3,2017-06-28,197.7,52.4,3.93617
4,2017-06-29,192.6,45.2,3.83577


In [5]:
# Calculate total ICE (Internal Combustion Engine) miles driven
voltDailyData = voltDailyData.rename(columns = {'MilesDriven':'TotalMilesDriven'})
voltDailyData['TotalICEMiles'] = voltDailyData['TotalMilesDriven'] - voltDailyData['EvMilesDriven']
voltDailyData.head()

,Date,TotalMilesDriven,EvMilesDriven,GallonsBurned,TotalICEMiles
0,2017-06-25,163.7,52.6,3.03005,111.1
1,2017-06-26,202.4,42.5,4.15543,159.9
2,2017-06-27,202.3,53.8,3.97314,148.5
3,2017-06-28,197.7,52.4,3.93617,145.3
4,2017-06-29,192.6,45.2,3.83577,147.4


In [6]:
# Create new DailyMPG Column, round values after adding new column
voltDailyData['DailyMPG'] = voltDailyData['TotalMilesDriven'] / voltDailyData['GallonsBurned']
voltDailyData['DailyMPG'] = pd.to_numeric(voltDailyData['DailyMPG'], errors='coerse')
voltDailyData = voltDailyData.replace(np.nan, 0)
voltDailyData = voltDailyData.round(2)
voltDailyData.head()

,Date,TotalMilesDriven,EvMilesDriven,GallonsBurned,TotalICEMiles,DailyMPG
0,2017-06-25,163.7,52.6,3.03,111.1,54.03
1,2017-06-26,202.4,42.5,4.16,159.9,48.71
2,2017-06-27,202.3,53.8,3.97,148.5,50.92
3,2017-06-28,197.7,52.4,3.94,145.3,50.23
4,2017-06-29,192.6,45.2,3.84,147.4,50.21


In [7]:
# Add '250+' miles to any DailyMPG value that has a value of infinity
# Why 250+? That's the "infinity mileage" that the volt displays in car
voltDailyData = voltDailyData.replace(np.inf, 250)
voltDailyData.head()

,Date,TotalMilesDriven,EvMilesDriven,GallonsBurned,TotalICEMiles,DailyMPG
0,2017-06-25,163.7,52.6,3.03,111.1,54.03
1,2017-06-26,202.4,42.5,4.16,159.9,48.71
2,2017-06-27,202.3,53.8,3.97,148.5,50.92
3,2017-06-28,197.7,52.4,3.94,145.3,50.23
4,2017-06-29,192.6,45.2,3.84,147.4,50.21


In [8]:
voltDailyData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118 entries, 0 to 117
Data columns (total 6 columns):
Date                118 non-null object
TotalMilesDriven    118 non-null float64
EvMilesDriven       118 non-null float64
GallonsBurned       118 non-null float64
TotalICEMiles       118 non-null float64
DailyMPG            118 non-null float64
dtypes: float64(5), object(1)
memory usage: 5.6+ KB


## Volt Readings

In [9]:
#Volt Readings throughout the day (usually two), convert Timestamp to datetime object
voltReadings = pd.read_csv('data/voltdata_' + currentDate.strftime('%Y%m%d') + '.csv')
voltReadings.rename(columns = {'LocalTimestamp':'Date', 'GallonsBurned': 'LifeTimeGalBurned'}, inplace=True)
voltReadings['Date'] = pd.to_datetime(voltReadings['Date'])
voltReadings.head()

,Date,BatteryLevel,ChargeState,OilLife,TirePressure_RF,TirePressure_RR,TirePressure_LF,TirePressure_LR,LifeTimeGalBurned,LifetimeFuelEcon,LifetimeEvMiles,LifetimeMiles,LastTripFuelEcon,LastTripEvMiles,LastTripMiles,GallonsOfGas
0,2018-02-22 01:03:52,0.0,NaN,0.0,0.0,0.0,0.0,0.0,865.76297,62.69393,22069.13492,54278.08454,0.0,0.0,0.0,4.66528
1,2018-02-21 13:05:27,0.0,NaN,0.0,0.0,0.0,0.0,0.0,865.76297,62.69393,22069.13492,54278.08454,0.0,0.0,0.0,4.66528
2,2018-02-21 01:01:29,0.0,NaN,0.0,0.0,0.0,0.0,0.0,865.76297,62.69393,22069.13492,54278.08454,0.0,0.0,0.0,4.66528
3,2018-02-20 13:02:28,0.0,NaN,0.0,0.0,0.0,0.0,0.0,863.32202,62.75611,22069.13492,54178.73350,0.0,0.0,0.0,7.00320
4,2018-02-20 01:13:54,0.0,NaN,0.0,0.0,0.0,0.0,0.0,862.58762,62.78209,22069.13492,54155.05304,0.0,0.0,0.0,7.62665


In [10]:
# Select only columns desired
voltReadings = voltReadings[['Date', 'LifeTimeGalBurned', 'LifetimeFuelEcon', 'LifetimeEvMiles','LifetimeMiles']]

In [11]:
# Calculate LifetimeICEMiles
voltReadings['LifetimeICEMiles'] = voltReadings['LifetimeMiles'] - voltReadings['LifetimeEvMiles']

In [12]:
# Sort based on timestamp, reorder columns, round values
voltReadings.sort_values(by='Date', inplace=True)
voltReadings = voltReadings[['Date', 'LifeTimeGalBurned','LifetimeFuelEcon','LifetimeEvMiles','LifetimeICEMiles','LifetimeMiles']]
voltReadings = voltReadings.round(2)
voltReadings['Date'] = voltReadings.Date.dt.normalize()
voltReadings.head()

,Date,LifeTimeGalBurned,LifetimeFuelEcon,LifetimeEvMiles,LifetimeICEMiles,LifetimeMiles
282,2017-06-24,593.83,68.35,17941.91,22643.54,40585.45
281,2017-06-25,593.83,68.35,17941.91,22643.54,40585.45
280,2017-06-25,596.65,68.27,17988.50,22746.30,40734.80
279,2017-06-26,596.86,68.27,17994.46,22754.70,40749.16
278,2017-06-26,601.02,68.14,18036.93,22914.62,40951.55


In [13]:
# Drop dupes based on date column
voltReadings.drop_duplicates(subset='Date', keep='last', inplace=True)
voltReadings.head()

,Date,LifeTimeGalBurned,LifetimeFuelEcon,LifetimeEvMiles,LifetimeICEMiles,LifetimeMiles
282,2017-06-24,593.83,68.35,17941.91,22643.54,40585.45
280,2017-06-25,596.65,68.27,17988.50,22746.30,40734.80
278,2017-06-26,601.02,68.14,18036.93,22914.62,40951.55
276,2017-06-27,604.19,68.05,18084.05,23032.74,41116.79
274,2017-06-28,608.17,67.93,18130.94,23180.56,41311.50


In [14]:
# Combine dataframes based on date indexes
voltStats = pd.concat([voltReadings.set_index('Date'),voltDailyData.set_index('Date')], join='inner', axis=1)
voltStats.tail()

,LifeTimeGalBurned,LifetimeFuelEcon,LifetimeEvMiles,LifetimeICEMiles,LifetimeMiles,TotalMilesDriven,EvMilesDriven,GallonsBurned,TotalICEMiles,DailyMPG
Date,,,,,,,,,,
2018-02-17,859.54,62.84,22018.46,31995.01,54013.47,20.1,20.1,0.13,0.0,155.28
2018-02-18,859.67,62.85,22038.57,31995.02,54033.59,0.0,0.0,0.00,0.0,0.00
2018-02-19,862.13,62.79,22062.27,32069.85,54132.12,121.5,30.6,2.92,90.9,41.58
2018-02-20,863.32,62.76,22069.13,32109.60,54178.73,123.0,0.0,3.18,123.0,38.74
2018-02-21,865.76,62.69,22069.13,32208.95,54278.08,0.0,0.0,0.00,0.0,0.00


In [15]:
#Remove rows that show 0 for TotalMilesDriven, EvMilesDriven, GallonsBurned, TotalICEMiles, DailyMPG (Days not driven)
voltStats = voltStats[voltStats.DailyMPG != 0]
voltStats.tail()

,LifeTimeGalBurned,LifetimeFuelEcon,LifetimeEvMiles,LifetimeICEMiles,LifetimeMiles,TotalMilesDriven,EvMilesDriven,GallonsBurned,TotalICEMiles,DailyMPG
Date,,,,,,,,,,
2018-02-13,852.22,63.03,21976.53,31740.42,53716.95,122.9,0.0,3.39,122.9,36.29
2018-02-16,857.05,62.91,22004.64,31910.39,53915.03,196.9,41.9,4.67,155.0,42.16
2018-02-17,859.54,62.84,22018.46,31995.01,54013.47,20.1,20.1,0.13,0.0,155.28
2018-02-19,862.13,62.79,22062.27,32069.85,54132.12,121.5,30.6,2.92,90.9,41.58
2018-02-20,863.32,62.76,22069.13,32109.60,54178.73,123.0,0.0,3.18,123.0,38.74


In [16]:
# Check info on columns
voltStats.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 75 entries, 2017-06-25 to 2018-02-20
Data columns (total 10 columns):
LifeTimeGalBurned    75 non-null float64
LifetimeFuelEcon     75 non-null float64
LifetimeEvMiles      75 non-null float64
LifetimeICEMiles     75 non-null float64
LifetimeMiles        75 non-null float64
TotalMilesDriven     75 non-null float64
EvMilesDriven        75 non-null float64
GallonsBurned        75 non-null float64
TotalICEMiles        75 non-null float64
DailyMPG             75 non-null float64
dtypes: float64(10)
memory usage: 6.4 KB


In [17]:
# Write voltStats DF to CSV
voltStats.to_csv('output/voltstats.csv', encoding='utf-8')

In [18]:
# Create Excel writer using XlsxWriter as the engine.
#writer = pd.ExcelWriter('output/voltstats.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
#voltStats.to_excel(writer, sheet_name='Sheet1')

# Upload a CSV to S3
data = open('output/voltstats.csv', 'rb')
s3.Bucket('volt-metrics').put_object(Key='voltstats.csv', Body=data)

# Close the Pandas Excel writer and output the Excel file.
#writer.save()

s3.Object(bucket_name='volt-metrics', key='voltstats.csv')